# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
df = pd.read_csv (r'C:\Users\212760357\Data_Bootcamp_Python\python-api-challenge\WeatherPy\my_weather_data.csv') 
df.head()


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Atuona,-9.8000,-139.0333,82.60,71,60,12.24,PF,1587060000
1,1,Busselton,-33.6500,115.3333,56.79,79,0,11.56,AU,1587060000
2,2,Ushuaia,-54.8000,-68.3000,56.21,53,99,6.78,AR,1587060000
3,3,Leningradskiy,69.3833,178.4167,13.68,95,100,9.44,RU,1587060000
4,4,Sitka,57.0531,-135.3300,44.96,90,98,4.03,US,1587060000


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
gmaps.configure(api_key=g_key)


In [26]:
locations = df[["Latitude", "Longitude"]].astype(float)
humidity = df["Humidity"].astype(float)

fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heatmap = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heatmap)

plt.savefig("vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
criteria = pd.DataFrame(df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (criteria["Max Temp"] <= 82) & (criteria["Max Temp"] > 72)
wind_speed = criteria["Wind Speed"] < 15
cloudiness = criteria["Cloudiness"] == 0
criteria[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
81,Makkah al Mukarramah,72.88,4.56,0
89,Sitrah,74.61,3.76,0
249,Gobabis,74.73,4.32,0
275,Mocuba,76.39,6.67,0
340,Buin,81.57,7.00,0
348,Alegrete,72.61,6.82,0
373,Doha,74.62,2.86,0
509,Umm Kaddadah,78.55,12.53,0
531,Reconquista,76.24,4.14,0
558,Mount Isa,77.50,8.50,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
hotel_df = pd.DataFrame(df, columns=["City", "Country", "Latitude", "Longitude"])

hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,Atuona,PF,-9.8000,-139.0333,
1,Busselton,AU,-33.6500,115.3333,
2,Ushuaia,AR,-54.8000,-68.3000,
3,Leningradskiy,RU,69.3833,178.4167,
4,Sitka,US,57.0531,-135.3300,
...,...,...,...,...,...
577,Burgeo,CA,47.6167,-57.6152,
578,Kikwit,CD,-5.0410,18.8162,
579,Beloha,MG,-25.1667,45.0500,
580,Shunyi,CN,40.1250,116.6453,


In [32]:
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [34]:
hotel_name
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()
hotel_df.to_csv('hotel_export.csv')

In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [38]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))